<a href="https://colab.research.google.com/github/mark910-ai/FlagEmbedding/blob/master/EmbeddingSearchValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!git clone https://github.com/mark910-ai/FlagEmbedding.git

Cloning into 'FlagEmbedding'...
remote: Enumerating objects: 10056, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 10056 (delta 51), reused 56 (delta 32), pack-reused 9961 (from 2)
Receiving objects: 100% (10056/10056), 54.30 MiB | 28.15 MiB/s, done.
Resolving deltas: 100% (5565/5565), done.
Updating files: 100% (1000/1000), done.


In [2]:
!pip install -U FlagEmbedding

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

sentences_1 = ["In this sentence: In the case of a limited company, the secretary or a director should sign the form., what is the meaning of the word: limited company?"]
sentences_2 = ["The word limited company is a ['noun'] and its definition is: a British company whose owners have to pay only a limited amount if the company gets into debt."]
sentences_3 = ["The word liturgy is a ['noun'] and its definition is: a way of praying in a religious service using a fixed order of words, prayers etc."]

embeddings_1 = model.encode(sentences_1,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model_path = '/content/drive/MyDrive/DSSM/encoder_only_m3_bge-m3_sd/checkpoint-10000'

model = BGEM3FlagModel(model_path,
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

sentences_1 = ["In this sentence: In the case of a limited company, the secretary or a director should sign the form., what is the meaning of the word: limited company?"]
sentences_2 = ["The word limited company is a ['noun'] and its definition is: a British company whose owners have to pay only a limited amount if the company gets into debt."]
sentences_3 = ["The word liturgy is a ['noun'] and its definition is: a way of praying in a religious service using a fixed order of words, prayers etc."]

embeddings_1 = model.encode(sentences_1,
                            batch_size=12,
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[[0.8564]]


In [3]:
import json
from typing import Dict, List

def load_jsonl(file_path: str, limit: int = None) -> List[Dict]:
    """
    load jsonl file
    Args:
        file_path: jsonl file path
        limit: limit the number of lines to read, default is None to read all
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if line.strip():
                data.append(json.loads(line))
                if limit and i >= limit - 1:
                    break
    return data

In [4]:
def inference(data: List[Dict]):
    from FlagEmbedding import BGEM3FlagModel

    # model_path = '/content/drive/MyDrive/DSSM/encoder_only_m3_bge-m3_sd/checkpoint-10000'
    model_path = 'BAAI/bge-m3'
    # 0.63
    # model_path = 'BAAI/bge-en-icl'
    # 0.53
    # model_path = 'BAAI/bge-large-en-v1.5'
    # 0.63

    model = BGEM3FlagModel(model_path,
                        query_instruction_for_retrieval="Given an english sentence and a word explanation, determine if the word explanation is correct or not in the sentence context.",
                        use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

    sentences_1 = [item['query'] for item in data]
    sentences_2 = [item['document'] for item in data]

    embeddings_1 = model.encode(sentences_1,
                                batch_size=12,
                                max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                                )['dense_vecs']
    embeddings_2 = model.encode(sentences_2)['dense_vecs']
    similarity = embeddings_1 @ embeddings_2.T
    # Get diagonal elements which represent similarities between corresponding pairs
    similarity_scores = similarity.diagonal()
    return similarity_scores

In [9]:
import random
data = load_jsonl('/content/FlagEmbedding/dataset/gse_train_data/data/validation_data_all.jsonl')
print(len(data))
# shuffle data
random.shuffle(data)
data = data[:100]
similarity = inference(data)
threshold = 0.5
# compare similarity with label and print the accuracy
# also print the false positive and false negative, and the precision and recall
precision = 0
recall = 0
false_positive = 0
false_negative = 0
correct = 0
for i, item in enumerate(data):
    if similarity[i] > threshold and item['label'] == 1:
        correct += 1
    elif similarity[i] <= threshold and item['label'] == 0:
        correct += 1
    elif similarity[i] > threshold and item['label'] == 0:
        false_positive += 1
    elif similarity[i] <= threshold and item['label'] == 1:
        false_negative += 1
precision = correct / (correct + false_positive)
recall = correct / (correct + false_negative)
print(f'Accuracy: {correct / len(data)}')
print(f'False Positive: {false_positive}')
print(f'False Negative: {false_negative}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

8062


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Accuracy: 0.9
False Positive: 4
False Negative: 6
Precision: 0.9574468085106383
Recall: 0.9375
